## Generating Synthetic Test Datasets

**Why use Synthetic Test Datasets?**

Evaluating the performance of RAG (Retrieval-Augmented Generation) augmented pipelines is crucial.

However, manually creating hundreds of QA (Question-Answer-Context) samples from documents can be time-consuming and labor-intensive. Additionally, human-generated questions may struggle to reach the level of complexity needed for thorough evaluation, ultimately affecting the quality of the assessment.

Using synthetic data generation can reduce developer time in the data aggregation process **by up to 90%**.


In [3]:
! pip install pdfplumber


In [4]:
import pdfplumber
from typing import List, Dict, Any

class PDFLoader:
    def __init__(self, file_path: str, start_page: int = None, end_page: int = None):
        self.file_path = file_path
        self.start_page = start_page
        self.end_page = end_page

    def load(self) -> Dict[str, Any]:
        combined_text = ""
        metadata = {}

        with pdfplumber.open(self.file_path) as pdf:
            total_pages = len(pdf.pages)

            start = (self.start_page or 1) - 1
            end = min(self.end_page or total_pages, total_pages)

            for page_num in range(start, end):
                page = pdf.pages[page_num]
                text = page.extract_text()
                combined_text += text + "\n"

            metadata = {
                "source": self.file_path,
                "filename": self.file_path,
                "total_pages": total_pages,
                "extracted_pages": f"{start + 1}-{end}"
            }

            for key, value in pdf.metadata.items():
                if isinstance(value, (str, int)):
                    metadata[key] = value

        return {
            "page_content": combined_text.strip(),
            "metadata": metadata
        }

## Document Used for Practice

Amazon Bedrock Manual Documentation (https://docs.aws.amazon.com/bedrock/latest/userguide/)

- Link: https://d1jp7kj5nqor8j.cloudfront.net/bedrock-manual.pdf
- File name: `bedrock-manual.pdf`

_Please copy the downloaded file to the data folder for the practice session_

In [11]:
loader = PDFLoader("../data/241231-Sonnet-recipe.pdf", start_page=1, end_page=3)
docs = loader.load()

In [12]:
docs

{'page_content': '볶음요리로는 먼저 김치볶음밥이 있는데, 잘 익은 김치를 잘게 썰어 밥과 함께\n볶다가 마지막에 참기름을 둘러 고소한 향을 더합니다. 제육볶음은 돼지고기를 고\n추장 양념에 버무려 매콤달콤하게 볶아내며, 양파와 당근 등 채소를 함께 넣어\n영양을 높입니다. 낙지볶음은 신선한 낙지를 손질해 고추장 양념에 볶아 매콤하게\n만드는 요리입니다.\n양식으로는 크림파스타가 대표적인데, 생크림과 마늘, 양파를 볶아 소스를 만들고\n베이컨이나 새우를 더해 고소하게 완성합니다. 토마토파스타는 토마토소스에 마늘\n과 양파를 볶아 넣고 바질을 곁들여 상큼한 맛을 냅니다. 미트소스스파게티는 다\n진 쇠고기를 토마토소스와 함께 오래 끓여 깊은 맛을 만듭니다.\n일식에서는 돈카츠가 인기 있는 메뉴로, 돼지고기를 얇게 펴서 빵가루를 묻혀 바\n삭하게 튀겨내고 특제 소스를 곁들입니다. 카레라이스는 당근, 감자, 양파를 카\n레가루와 함께 끓여 걸쭉한 소스를 만들어 밥에 부어 먹습니다. 오야코동은 닭고\n기와 달걀을 간장 베이스의 달달한 육수에 조려 밥 위에 올립니다.\n중식으로는 마파두부가 있는데, 두부와 다진 고기를 매운 두반장 소스에 볶아 만\n듭니다. 깐풍기는 닭고기를 튀겨서 매콤달콤한 소스를 絡めて만드는 요리이고, 탕\n수육은 돼지고기를 튀겨서 새콤달콤한 소스를 부어 먹습니다.\n태국 요리로는 팟타이가 유명한데, 쌀국수를 새우와 함께 볶고 땅콩가루를 뿌려\n고소한 맛을 냅니다. 그린커리는 코코넛밀크와 향신료를 넣어 만든 커리에 닭고기\n와 가지를 넣어 끓입니다. 톰얌쿵은 새우와 버섯을 레몬그라스, 카피르라임 등의\n향신료와 함께 시큼매콤하게 끓이는 수프입니다.\n베트남 요리에서는 쌀국수인 포가 대표적인데, 소고기 육수에 쌀국수와 고기를 넣\n고 고수, 숙주나물을 곁들여 먹습니다. 반미는 바게트빵에 각종 채소와 고기를\n넣어 만드는 샌드위치이며, 분짜는 구운 돼지고기와 쌀국수를 느억맘 소스에 찍어\n먹는 요리입니다.\n이탈리아 요리로 더 들어가보면, 리

## Document Preprocessing

In [13]:
import re
from typing import List, Optional

def split_text(text: str, chunk_size: int = 1000, chunk_overlap: int = 100, separators: Optional[List[str]] = None) -> List[str]:
    separators = separators or ["\n\n", "\n", " ", ""]

    def _split_text_recursive(text: str, separators: List[str]) -> List[str]:
        if not separators:
            return [text]

        separator = separators[0]
        splits = re.split(f"({re.escape(separator)})", text)
        splits = ["".join(splits[i:i+2]) for i in range(0, len(splits), 2)]

        final_chunks = []
        current_chunk = ""

        for split in splits:
            if len(current_chunk) + len(split) <= chunk_size:
                current_chunk += split
            else:
                if current_chunk:
                    final_chunks.append(current_chunk)
                if len(split) > chunk_size:
                    subsplits = _split_text_recursive(split, separators[1:])
                    final_chunks.extend(subsplits)
                else:
                    current_chunk = split

        if current_chunk:
            final_chunks.append(current_chunk)

        return final_chunks

    chunks = _split_text_recursive(text, separators)

    if chunk_overlap > 0:
        overlapped_chunks = []
        for i, chunk in enumerate(chunks):
            if i == 0:
                overlapped_chunks.append(chunk)
            else:
                overlap_text = chunks[i-1][-chunk_overlap:]
                overlapped_chunks.append(overlap_text + chunk)
        chunks = overlapped_chunks

    return chunks

In [14]:
chunks = split_text(docs['page_content'], 1000, 0)
len(chunks)

4

In [15]:
chunks_with_metadata = []
for i, chunk in enumerate(chunks):
    chunks_with_metadata.append({
        'content': chunk,
        'metadata': {
            'chunk_id': i,
            'filename': docs['metadata'].get('filename', 'unknown')
        }
    })

In [16]:
chunks_with_metadata[0]

{'content': '볶음요리로는 먼저 김치볶음밥이 있는데, 잘 익은 김치를 잘게 썰어 밥과 함께\n볶다가 마지막에 참기름을 둘러 고소한 향을 더합니다. 제육볶음은 돼지고기를 고\n추장 양념에 버무려 매콤달콤하게 볶아내며, 양파와 당근 등 채소를 함께 넣어\n영양을 높입니다. 낙지볶음은 신선한 낙지를 손질해 고추장 양념에 볶아 매콤하게\n만드는 요리입니다.\n양식으로는 크림파스타가 대표적인데, 생크림과 마늘, 양파를 볶아 소스를 만들고\n베이컨이나 새우를 더해 고소하게 완성합니다. 토마토파스타는 토마토소스에 마늘\n과 양파를 볶아 넣고 바질을 곁들여 상큼한 맛을 냅니다. 미트소스스파게티는 다\n진 쇠고기를 토마토소스와 함께 오래 끓여 깊은 맛을 만듭니다.\n일식에서는 돈카츠가 인기 있는 메뉴로, 돼지고기를 얇게 펴서 빵가루를 묻혀 바\n삭하게 튀겨내고 특제 소스를 곁들입니다. 카레라이스는 당근, 감자, 양파를 카\n레가루와 함께 끓여 걸쭉한 소스를 만들어 밥에 부어 먹습니다. 오야코동은 닭고\n기와 달걀을 간장 베이스의 달달한 육수에 조려 밥 위에 올립니다.\n중식으로는 마파두부가 있는데, 두부와 다진 고기를 매운 두반장 소스에 볶아 만\n듭니다. 깐풍기는 닭고기를 튀겨서 매콤달콤한 소스를 絡めて만드는 요리이고, 탕\n수육은 돼지고기를 튀겨서 새콤달콤한 소스를 부어 먹습니다.\n태국 요리로는 팟타이가 유명한데, 쌀국수를 새우와 함께 볶고 땅콩가루를 뿌려\n고소한 맛을 냅니다. 그린커리는 코코넛밀크와 향신료를 넣어 만든 커리에 닭고기\n와 가지를 넣어 끓입니다. 톰얌쿵은 새우와 버섯을 레몬그라스, 카피르라임 등의\n향신료와 함께 시큼매콤하게 끓이는 수프입니다.\n베트남 요리에서는 쌀국수인 포가 대표적인데, 소고기 육수에 쌀국수와 고기를 넣\n고 고수, 숙주나물을 곁들여 먹습니다. 반미는 바게트빵에 각종 채소와 고기를\n넣어 만드는 샌드위치이며, 분짜는 구운 돼지고기와 쌀국수를 느억맘 소스에 찍어\n먹는 요리입니다.\n이탈리아 요리로 더 들어가보면, 리소토는 아

## Test Q&A Dataset Generation

In [17]:
import boto3
from botocore.config import Config

region = 'us-west-2'
retry_config = Config(
    region_name=region,
    retries={"max_attempts": 10, "mode": "standard"}
)
boto3_client = boto3.client("bedrock-runtime", region_name=region, config=retry_config)

In [18]:
import random
import json
from time import sleep

def converse_with_bedrock(model_id, sys_prompt, usr_prompt):
    temperature = 0.5
    top_p = 0.9
    inference_config = {"temperature": temperature, "topP": top_p}
    response = boto3_client.converse(
        modelId=model_id,
        messages=usr_prompt, 
        system=sys_prompt,
        inferenceConfig=inference_config,
    )
    return response

def create_prompt(sys_template, user_template):
    sys_prompt = [{"text": sys_template}]
    usr_prompt = [{"role": "user", "content": [{"text": user_template}]}]
    return sys_prompt, usr_prompt

def get_context_chunks(chunks_with_metadata, start_id):
    context_chunks = [
        chunks_with_metadata[start_id]['content'],
        chunks_with_metadata[start_id + 1]['content'],
        chunks_with_metadata[start_id + 2]['content']
    ]
    return " ".join(context_chunks)

### Tool Use 

LLM will generate Q&A dataset that conforms to the schema description in the tooluse config.

In [19]:
tool_config = {
    "tools": [
        {
            "toolSpec": {
                "name": "QuestionAnswerGenerator",
                "description": "Generates questions and answers based on the given context.",
                "inputSchema": {
                    "json": {
                        "type": "object",
                        "properties": {
                            "question": {
                                "type": "string",
                                "description": "The generated question"
                            },
                            "answer": {
                                "type": "string",
                                "description": "The answer to the generated question"
                            }
                        },
                        "required": ["question", "answer"]
                    }
                }
            }
        }
    ]
}

In [20]:
def converse_with_bedrock_tools(sys_prompt, usr_prompt, tool_config):
    temperature = 0.0
    top_p = 0.1
    top_k = 1
    inference_config = {"temperature": temperature, "topP": top_p}
    additional_model_fields = {"top_k": top_k}
    response = boto3_client.converse(
        modelId="anthropic.claude-3-5-sonnet-20240620-v1:0",
        messages=usr_prompt,
        system=sys_prompt,
        inferenceConfig=inference_config,
        additionalModelRequestFields=additional_model_fields,
        toolConfig=tool_config
    )
    return response

def parse_tool_use(message):
    stop_reason = message['stopReason']

    if stop_reason == 'tool_use':
        tool_requests = message['output']['message']['content']
        for tool_request in tool_requests:
            if 'toolUse' in tool_request:
                tool = tool_request['toolUse']

                if tool['name'] == 'QuestionAnswerGenerator':
                    return tool['input']
    return None

## Q&A Dataset Generation Instruction

- `simple`: directly answerable questions from the given context
- `complex`: reasoning questions and answers.

_Modify the system/user prompts tailored to your dataset_

Generated Q&A pair will be stored in `data/qa_dataset.jsonl`

In [29]:
def generate_qa_dataset(chunks, num_pairs=5, output_file="../data/sample_qa_dataset.jsonl"):
    total_chunks = len(chunks)
    dataset = []

    for i in range(num_pairs):
        start_id = random.randint(0, total_chunks - 3)
        context = get_context_chunks(chunks_with_metadata, start_id)

        if i % 2 == 0:
            sys_template = """
            You are an expert at generating practical questions based on given documentation.
            Your task is to generate complex, reasoning questions and answers in korean.

            Follow these rules:
            1. Generate questions that reflect real user information needs related to the document's subject matter (e.g., technical docs : feature availability, implementation details)
            2. Ensure questions are relevant, concise, preferably under 25 words, and fully answerable with the provided information
            3. Focus on extracting key information that users are likely to seek, while avoiding narrow or less important questions.
            4. When provided with code blocks, focus on understanding the overall functionality rather than the specific syntax or variables. Feel free to request examples of how to use key APIs or features.
            5. Do not use phrases like 'based on the provided context' or 'according to the context'.
            """
            question_type = "complex"
        else:
            sys_template = """
            You are an expert at generating practical questions based on given documentation.
            Your task is to create simple, directly answerable questions from the given context in korean.

            Follow these rules:
            1. Generate questions that reflect real user information needs related to the document's subject matter (e.g., technical docs : feature availability, implementation details)
            2. Ensure questions are relevant, concise, preferably under 10 words, and fully answerable with the provided information
            3. Focus on extracting key information that users are likely to seek, while avoiding narrow or less important questions.
            4. When provided with code blocks, focus on understanding the overall functionality rather than the specific syntax or variables. Feel free to request examples of how to use key APIs or features.
            5. Do not use phrases like 'based on the provided context' or 'according to the context'.
            """
            question_type = "simple"

        user_template = f"""
        Generate a {question_type} question and its answer based on the following context:

        Context: {context}

        Use the QuestionAnswerGenerator tool to provide the output.
        """

        sys_prompt, user_prompt = create_prompt(sys_template, user_template)
        response = converse_with_bedrock_tools(sys_prompt, user_prompt, tool_config)
        qa_data = parse_tool_use(response)

        if qa_data:
            qa_item = {
                "question": qa_data["question"],
                "ground_truth": qa_data["answer"],
                "question_type": question_type,
                "contexts": context
            }

            print(qa_item)

            with open(output_file, 'a') as f:
                json.dump(qa_item, f)
                f.write('\n')

            dataset.append(qa_item)

        sleep(5)

    return dataset

In [30]:
generate_qa_dataset(chunks_with_metadata, 3)

{'question': '한국, 이탈리아, 태국의 대표적인 요리들을 각각 하나씩 선택하여, 이 요리들의 주요 재료와 조리 방법의 유사점과 차이점을 비교 분석해보세요. 특히 각 요리의 문화적 특성이 어떻게 반영되어 있는지 설명해주세요.', 'ground_truth': '한국의 김치볶음밥, 이탈리아의 리소토, 태국의 팟타이를 비교해보겠습니다.\n\n1. 주요 재료:\n- 김치볶음밥: 밥, 김치, 참기름\n- 리소토: 아르보리오 쌀, 화이트와인, 버터, 육수\n- 팟타이: 쌀국수, 새우, 땅콩가루\n\n2. 조리 방법:\n유사점: 세 요리 모두 볶는 조리법을 사용합니다.\n차이점: \n- 김치볶음밥: 빠르게 고열로 볶아 만듭니다.\n- 리소토: 천천히 육수를 넣어가며 졸이듯이 조리합니다.\n- 팟타이: 중간 불에서 재료를 순서대로 넣어가며 볶습니다.\n\n3. 문화적 특성:\n- 김치볶음밥: 한국의 발효 문화를 대표하는 김치를 활용하여 간편하게 만드는 요리로, 한국인의 실용적이고 빠른 식문화를 반영합니다.\n- 리소토: 이탈리아의 여유로운 식문화를 반영하여 천천히 정성을 들여 만듭니다. 와인을 사용하는 것은 이탈리아의 와인 문화를 보여줍니다.\n- 팟타이: 태국의 길거리 음식 문화를 대표하며, 다양한 재료와 맛(새콤, 달콤, 고소함)의 조화를 중시하는 태국 요리의 특징을 잘 보여줍니다.\n\n이처럼 세 요리는 모두 볶음 요리이지만, 각 나라의 고유한 식재료와 조리 방식, 그리고 식문화적 특성을 뚜렷하게 반영하고 있습니다.', 'question_type': 'complex', 'contexts': '볶음요리로는 먼저 김치볶음밥이 있는데, 잘 익은 김치를 잘게 썰어 밥과 함께\n볶다가 마지막에 참기름을 둘러 고소한 향을 더합니다. 제육볶음은 돼지고기를 고\n추장 양념에 버무려 매콤달콤하게 볶아내며, 양파와 당근 등 채소를 함께 넣어\n영양을 높입니다. 낙지볶음은 신선한 낙지를 손질해 고추장 양념에 볶아 매콤하게\n만드는 요리입니다.\n양식으로는 크림파스타가 대표적인데,

[{'question': '한국, 이탈리아, 태국의 대표적인 요리들을 각각 하나씩 선택하여, 이 요리들의 주요 재료와 조리 방법의 유사점과 차이점을 비교 분석해보세요. 특히 각 요리의 문화적 특성이 어떻게 반영되어 있는지 설명해주세요.',
  'ground_truth': '한국의 김치볶음밥, 이탈리아의 리소토, 태국의 팟타이를 비교해보겠습니다.\n\n1. 주요 재료:\n- 김치볶음밥: 밥, 김치, 참기름\n- 리소토: 아르보리오 쌀, 화이트와인, 버터, 육수\n- 팟타이: 쌀국수, 새우, 땅콩가루\n\n2. 조리 방법:\n유사점: 세 요리 모두 볶는 조리법을 사용합니다.\n차이점: \n- 김치볶음밥: 빠르게 고열로 볶아 만듭니다.\n- 리소토: 천천히 육수를 넣어가며 졸이듯이 조리합니다.\n- 팟타이: 중간 불에서 재료를 순서대로 넣어가며 볶습니다.\n\n3. 문화적 특성:\n- 김치볶음밥: 한국의 발효 문화를 대표하는 김치를 활용하여 간편하게 만드는 요리로, 한국인의 실용적이고 빠른 식문화를 반영합니다.\n- 리소토: 이탈리아의 여유로운 식문화를 반영하여 천천히 정성을 들여 만듭니다. 와인을 사용하는 것은 이탈리아의 와인 문화를 보여줍니다.\n- 팟타이: 태국의 길거리 음식 문화를 대표하며, 다양한 재료와 맛(새콤, 달콤, 고소함)의 조화를 중시하는 태국 요리의 특징을 잘 보여줍니다.\n\n이처럼 세 요리는 모두 볶음 요리이지만, 각 나라의 고유한 식재료와 조리 방식, 그리고 식문화적 특성을 뚜렷하게 반영하고 있습니다.',
  'question_type': 'complex',
  'contexts': '볶음요리로는 먼저 김치볶음밥이 있는데, 잘 익은 김치를 잘게 썰어 밥과 함께\n볶다가 마지막에 참기름을 둘러 고소한 향을 더합니다. 제육볶음은 돼지고기를 고\n추장 양념에 버무려 매콤달콤하게 볶아내며, 양파와 당근 등 채소를 함께 넣어\n영양을 높입니다. 낙지볶음은 신선한 낙지를 손질해 고추장 양념에 볶아 매콤하게\n만드는 요리입니다.\n양식으로는 크림파스타가